In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
plt.style.use('ggplot')
%matplotlib inline

In [2]:
with open('Manual WorkItems alltypes Security.csv', encoding='ascii', errors='ignore') as infile:
    dataset1 = pd.read_csv(infile,names = ['Title', 'Security', 'SourceLink'])
with open('Manual WorkItems alltypes nonSecurity.csv', encoding='ascii', errors='ignore') as infile:
    dataset2 = pd.read_csv(infile,names = ['Title', 'Security', 'SourceLink'])

In [3]:
dataset1 = dataset1.sample(frac = 0.45, random_state= 123).reset_index(drop = True)
dataset1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63262 entries, 0 to 63261
Data columns (total 3 columns):
Title         63261 non-null object
Security      63262 non-null object
SourceLink    63262 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


In [4]:
dataset1['IsSecured'] = 1
dataset2['IsSecured'] = 0

In [5]:
dataset = pd.concat([dataset1, dataset2]).reset_index(drop = True)
dataset.head()

,Title,Security,SourceLink,IsSecured
0,[WDGISScan] Perf.Service.PASAPI 'pasdb.westus....,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1
1,[Studio]Form shouldn't load another datasource...,MS.Security,https://msazure.visualstudio.com/DefaultCollec...,1
2,Glusterfs should only allow certain ip range t...,MS.Security,https://msasg.visualstudio.com/DefaultCollecti...,1
3,[dev15->devmain] Fuzzing: Mac Word: doc: Proba...,MS.Security,http://bugcheck/bugs/OfficeMain/75542,1
4,(Port to wacserver 16) MSRC 32783. Word Crash ...,MS.Security,https://office.visualstudio.com/DefaultCollect...,1


In [6]:
dataset = dataset.sample(frac = 1).reset_index(drop = True)
dataset.head()

,Title,Security,SourceLink,IsSecured
0,"[ComboBox] The string ""Find Items"" of ComboBox...",MS.Security,https://msazure.visualstudio.com/DefaultCollec...,1
1,TVS: Warning 6262 in file: Xbox\base\net\core\...,MS.Security,https://microsoft.visualstudio.com/DefaultColl...,1
2,Investigate increase in S2S token duration,MS.Security,https://identitydivision.visualstudio.com/Defa...,1
3,[AWS] - Azure Email Service client certificate...,MS.Security,https://identitydivision.visualstudio.com/Defa...,1
4,[Screen Reader - Key Vault -Create] Narrator d...,MS.Accessibility,https://msazure.visualstudio.com/DefaultCollec...,0


In [7]:
dataset.dropna(axis=0, how='any', inplace = True)

In [8]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 122058 entries, 0 to 122059
Data columns (total 4 columns):
Title         122058 non-null object
Security      122058 non-null object
SourceLink    122058 non-null object
IsSecured     122058 non-null int64
dtypes: int64(1), object(3)
memory usage: 4.7+ MB


In [9]:
from nltk.corpus import stopwords
my_stopwords = stopwords.words('english')
type(my_stopwords)

list

In [10]:
from patsy import dmatrices
import statsmodels.discrete.discrete_model as sm1
from statsmodels.formula.api import logit, ols, poisson, probit
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer

In [11]:
import os, sys
def splitall(path):
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return " ".join(allparts)

In [12]:
#I will now apply the splitall function on the dataset.
dataset['Parsed Title'] = dataset['Title'].apply(splitall)
#dataset.head(100)

In [13]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
def clean_text(text):
    text = "".join([char if char.isalpha() else " " for char in text.lower()]).split()
    text = [ps.stem(word) for word in text]
    text = " ".join([word for word in text if (word not in my_stopwords and len(word) > 2)])
    return text.lower()

In [14]:
%time dataset['Clean Title'] = dataset['Parsed Title'].apply(clean_text)

Wall time: 42.5 s


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectPercentile, f_classif
X, y = dataset['Clean Title'], dataset['IsSecured']
tv = TfidfVectorizer(min_df=0.001, max_df= 1.0, use_idf=True,stop_words = 'english' )
#tv = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, stop_words = 'english')
#selector = SelectPercentile(f_classif, percentile = 10)
#selector.fit(tv.fit_transform(X))
X_train, X1, y_train, y1 = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X1, y1, test_size=0.5, random_state=42)
X_train = tv.fit_transform(X_train)#.toarray()
X_test = tv.transform(X_test)#.toarray()
X_val = tv.transform(X_val)#.toarray()
vocab = tv.get_feature_names()
# y_test = np.array(list(y_test))
# y_train = np.array(list(y_train))
# y_val = np.array(list(y_val))
print(len(vocab))


1105


In [16]:
X2 = pd.DataFrame(X_train.toarray(), columns = vocab)
X2["Intercept"] = 1.0

X3 = pd.DataFrame(X_val.toarray(), columns = vocab)
X3["Intercept"] = 1.0

X4 = pd.DataFrame(X_test.toarray(), columns = vocab)
X4["Intercept"] = 1.0

vocab = vocab +['Intercept']

In [17]:
X2.head()

,aad,abil,abl,acc,accept,access,account,aci,action,activ,...,write,wrong,xbox,xml,xss,year,zero,zone,zoom,Intercept
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
X2["IsSecured"] = list(y_train)

In [19]:
from pandas.core import datetools
import statsmodels.api as sm

C:\Users\inpganavar\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  """Entry point for launching an IPython kernel.


In [20]:
%time logit = sm.Logit(X2['IsSecured'], X2[vocab])

Wall time: 26.2 s


In [29]:
# fit the model
%time result = logit.fit(method='bfgs',maxiter=500)

Optimization terminated successfully.
         Current function value: 0.182705
         Iterations: 310
         Function evaluations: 312
         Gradient evaluations: 312
Wall time: 1min 45s


In [30]:
# cool enough to deserve it's own gist
result.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Results: Logit
==========================================================================
Model:                 Logit               Pseudo R-squared:    0.736     
Dependent Variable:    IsSecured           AIC:                 28972.4179
Date:                  2018-04-11 12:39    BIC:                 39149.1830
No. Observations:      73234               Log-Likelihood:      -13380.   
Df Model:              1105                LL-Null:             -50709.   
Df Residuals:          72128               Scale:               1.0000    
Converged:             1.0000                                             
--------------------------------------------------------------------------
                        Coef.   Std.Err.    z     P>|z|   [0.025   0.975] 
--------------------------------------------------------------------------
aad                     -0.3460   0.5518  -0.6271 0.5306  -1.4276   0.7355
abil                     0.0882   0.6101   0.1446 0.8850  -1.1076   1.2840
abl                      0.4301   0.4736   0.9083 0.3637  -0.4980   1.3583
acc                     -6.8093   1.5686  -4.3411 0.0000  -9.8836  -3.7350
accept                  -1.3926   0.5843  -2.3833 0.0172  -2.5379  -0.2474
access                  -7.4263   0.2475 -30.0015 0.0000  -7.9115  -6.9412
account                  1.2000   0.3072   3.9066 0.0001   0.5980   1.8020
aci                      2.6631   1.3844   1.9237 0.0544  -0.0503   5.3765
action                  -1.1635   0.4163  -2.7949 0.0052  -1.9794  -0.3476
activ                   -1.1994   0.3928  -3.0530 0.0023  -1.9693  -0.4294
actual                  -0.1888   0.9904  -0.1907 0.8488  -2.1300   1.7523
adapt                   -2.6696   0.8661  -3.0825 0.0021  -4.3670  -0.9722
add                     -0.2149   0.2272  -0.9462 0.3441  -0.6602   0.2303
addit                   -1.8163   0.5751  -3.1583 0.0016  -2.9434  -0.6891
address                  0.2261   0.4659   0.4853 0.6275  -0.6871   1.1393
adjust                   0.9918   0.6603   1.5021 0.1331  -0.3023   2.2859
admin                    0.8403   0.4071   2.0638 0.0390   0.0423   1.6382
administr                1.5963   0.8724   1.8297 0.0673  -0.1137   3.3062
adminport               11.9914   3.8362   3.1258 0.0018   4.4725  19.5103
advanc                   1.5565   0.6492   2.3976 0.0165   0.2841   2.8289
afd                     10.4902   3.5616   2.9454 0.0032   3.5096  17.4708
aft                     10.1660   3.6762   2.7654 0.0057   2.9608  17.3713
agent                   -0.7943   0.4460  -1.7810 0.0749  -1.6684   0.0798
aip                     -0.0010   0.8922  -0.0011 0.9991  -1.7497   1.7476
album                   -3.9554   1.1847  -3.3388 0.0008  -6.2773  -1.6335
alert                    0.5583   0.3734   1.4951 0.1349  -0.1736   1.2902
align                    0.0563   0.5298   0.1062 0.9154  -0.9821   1.0946
allow                    1.6324   0.4664   3.5001 0.0005   0.7183   2.5465
alreadi                  1.1776   0.8551   1.3771 0.1685  -0.4984   2.8536
alt                     -8.1317   1.1418  -7.1220 0.0000 -10.3695  -5.8939
altern                  -2.0166   0.8085  -2.4943 0.0126  -3.6012  -0.4320
alway                    0.7058   0.6430   1.0977 0.2723  -0.5545   1.9662
amazon                  -3.3309   3.2393  -1.0283 0.3038  -9.6798   3.0179
analysi                 -0.8013   0.4672  -1.7152 0.0863  -1.7170   0.1144
analyt                  -0.4318   0.6532  -0.6611 0.5086  -1.7121   0.8484
analyz                  -2.8352   0.8856  -3.2014 0.0014  -4.5709  -1.0994
android                  1.0564   0.3656   2.8896 0.0039   0.3399   1.7730
ani                     -0.3800   0.4509  -0.8429 0.3993  -1.2637   0.5036
announc                 -6.8299   0.7764  -8.7965 0.0000  -8.3517  -5.3081
anoth                    1.4117   0.7484   1.8862 0.0593  -0.0552   2.8786
answer                  -1.3298   0.7512  -1.7703 0.0767  -2.8020   0.1425
anyth 

In [31]:
#from pandas_ml import ConfusionMatrix
from sklearn.metrics import confusion_matrix

predict=result.predict(X3)

Y_val_predict=[]
for i in predict:
    if(i>=0.5):
        Y_val_predict.append(1)
    else:
        Y_val_predict.append(0)

conf=confusion_matrix(y_val,Y_val_predict)
#plt.imshow(conf, cmap='binary', interpolation='None')
print(conf)
#plt.show()

[[10875  1013]
 [  816 11708]]


In [34]:
print("odds ratio's")
print(np.exp(result.params))

odds ratio's
aad          7.074788e-01
abil         1.092232e+00
abl          1.537433e+00
acc          1.103442e-03
accept       2.484175e-01
access       5.953769e-04
account      3.320117e+00
aci          1.434086e+01
action       3.123950e-01
activ        3.013900e-01
actual       8.279313e-01
adapt        6.927891e-02
add          8.065923e-01
addit        1.626283e-01
address      1.253707e+00
adjust       2.696119e+00
admin        2.316947e+00
administr    4.934557e+00
adminport    1.613633e+05
advanc       4.742200e+00
afd          3.596198e+04
aft          2.600459e+04
agent        4.518969e-01
aip          9.989775e-01
album        1.915147e-02
alert        1.747724e+00
align        1.057869e+00
allow        5.116225e+00
alreadi      3.246576e+00
alt          2.940691e-04
                 ...     
way          1.233092e-01
wdg          5.131787e-01
wdgisscan    5.150152e+03
weak         1.244489e+03
web          3.912756e-01
webauth      6.555027e+07
websit       8.474559e-02

In [37]:
str("X")+str(range(0,10))

'Xrange(0, 10)'